# LlamaParse + Cortex Search

This notebook walks through how to parse a complex report with LlamaParse, how to load it in Snowflake, and how to create a RAG via Cortex Search on top of the data loaded into Snowflake.

## Set key and import libraries
For this example, you will need a LlamaCloud API key and a Snowflake account. Get LlamaCloud API key following these [instructions](https://docs.cloud.llamaindex.ai/api_key), and signup for Snowflake [here](https://signup.snowflake.com/).

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

# llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."

# snowflake
os.environ["SNOWFLAKE_ACCOUNT"] = "..." # note: "_" can cause problems with the connection, use "-" instead
os.environ["SNOWFLAKE_USER"] = "..."
os.environ["SNOWFLAKE_PASSWORD"] = "..."
os.environ["SNOWFLAKE_ROLE"] = "..."
os.environ["SNOWFLAKE_WAREHOUSE"] = "..."
os.environ["SNOWFLAKE_DATABASE"] = "..." # note: make sure to use a database that already
os.environ["SNOWFLAKE_SCHEMA"] = "..."

## Load Data

Use Snowflake's latest 10K or your favorite PDF.

To use Snowflake's latest 10K, download it from [here](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001640147/663fb935-b123-4bbb-8827-905bcbb8953c.pdf) and rename it as `snowflake_2025_10k.pdf`.

## Parse PDF

In [2]:
from llama_cloud_services import LlamaParse

parser = LlamaParse(
    num_workers=4,       # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",       # optionally define a language, default=en
)

# sync
result = parser.parse("./snowflake_2025_10k.pdf")

Started parsing the file under job_id bbd9b1ce-0a10-4699-874f-af0133189135


In [3]:
# get the llama-index markdown documents
markdown_documents = result.get_markdown_documents(split_by_page=True)

## Convert LlamaIndex Documents to Dataframe

In [4]:
import pandas as pd

def documents_to_dataframe(documents):
    rows = []
    for doc in documents:
        row = {}
        # Store document ID
        row["ID"] = doc.id_
        # Add all metadata items as separate columns
        for key, value in doc.metadata.items():
            row[key] = value
        # Get text from the text_resource attribute, if available
        row["text"] = getattr(doc.text_resource, "text", None)
        rows.append(row)
    return pd.DataFrame(rows)

In [5]:
documents_df = documents_to_dataframe(markdown_documents)

In [6]:
documents_df

,ID,page_number,file_name,text
0,5b98be7b-b051-4978-aa37-301a71ab7049,1,./snowflake_2025_10k.pdf,# Table of Contents\n\n# UNITED STATES\n\n# SE...
1,39beb5de-9853-4bc1-b2e0-443123e2e2e7,2,./snowflake_2025_10k.pdf,# TABLE OF CONTENTS\n\n| Special Note About Fo...
2,c7aa5c02-6440-42d8-9d4d-1aa933a9acc1,3,./snowflake_2025_10k.pdf,# Table of Contents\n\n# SPECIAL NOTE ABOUT FO...
3,1f558f93-c964-4a8d-bff2-7ed309b7bd10,4,./snowflake_2025_10k.pdf,# Table of Contents\n\nWe caution you that the...
4,a52294ba-613d-4475-8f8e-0a36879d05cc,5,./snowflake_2025_10k.pdf,# Table of Contents\n\n# SELECTED RISKS AFFECT...
...,...,...,...,...
224,727670de-e598-4648-8da7-36acb49e455e,225,./snowflake_2025_10k.pdf,# Exhibit 23.1\n\n# CONSENT OF INDEPENDENT REG...
225,813d21c2-7f9f-49b0-9b98-1e8d134c8e9b,226,./snowflake_2025_10k.pdf,# Exhibit 31.1\n\n# CERTIFICATION OF PRINCIPAL...
226,355b988e-9954-4c5e-95fa-98be39f8159f,227,./snowflake_2025_10k.pdf,# Exhibit 31.2\n\n# CERTIFICATION OF PRINCIPAL...
227,9e5d2584-624c-4155-9420-42b61ecb440f,228,./snowflake_2025_10k.pdf,# Exhibit 32.1\n\n# CERTIFICATION PURSUANT TO ...


## Write DataFrame to Snowflake table

In [7]:
from snowflake.snowpark import Session

# Create Snowpark session
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),            
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WARHEOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

session = Session.builder.configs(connection_parameters).create()

/opt/anaconda3/envs/oss_rag_stack/lib/python3.11/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (20.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
/opt/anaconda3/envs/oss_rag_stack/lib/python3.11/site-packages/snowflake/snowpark/session.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [8]:
# convert to Snowpark DataFrame
snowpark_df = session.create_dataframe(documents_df)

In [9]:
# Write Snowpark DataFrame to a Snowflake table
# Use 'overwrite' to replace table or 'append' to add to existing table
snowpark_df.write.mode("overwrite").save_as_table("snowflake_10k")

In [10]:
create_search_service_sql = """
CREATE OR REPLACE CORTEX SEARCH SERVICE SNOWFLAKE_10K_SEARCH_SERVICE
  ON TEXT
  ATTRIBUTES ID, PAGE_NUMBER, FILE_NAME
  WAREHOUSE = S
  TARGET_LAG = '1 hour'
    AS (
      SELECT
        ID,
        "page_number" AS PAGE_NUMBER,
        "file_name" AS FILE_NAME,
        "text" AS TEXT
      FROM SEC_10KS.PUBLIC.SNOWFLAKE_10K
      );
"""
session.sql(create_search_service_sql).collect()

[Row(status='Cortex search service SNOWFLAKE_10K_SEARCH_SERVICE successfully created.')]

In [11]:
import os
from snowflake.core import Root
from typing import List
from snowflake.snowpark.session import Session

class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 4):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(session)

        search_service = (root
          .databases["SEC_10KS"]
          .schemas["PUBLIC"]
          .cortex_search_services["SNOWFLAKE_10K_SEARCH_SERVICE"]
        )
        resp = search_service.search(
          query=query,
          columns=["text"],
          limit=self._limit_to_retrieve
        )

        if resp.results:
            return [curr["text"] for curr in resp.results]
        else:
            return []
        
retriever = CortexSearchRetriever(snowpark_session=session, limit_to_retrieve=5)

In [12]:
retrieved_context = retriever.retrieve(query="Who is the CRO of Snowflake?")

retrieved_context

['# Xksnowflake\n\n# March 4, 2025\n\n# Michael Gannon\n\nDear Michael,\n\nWe are excited to offer you the position of Chief Revenue Officer reporting to Sridhar Ramaswamy. You will work out of our office in Atlanta, Georgia.\n\n# Compensation and Benefits Information\n\nYour annual salary will be $500,000 per year, less taxes, payroll deductions and withholding. Our pay frequency is bi-weekly and you will receive your paycheck every other Friday (except if Friday falls on a holiday, then payday will be the day prior). You are eligible for benefits as set forth in Snowflake’s Employee Benefits Guide.\n\n# Annual Bonus\n\nYou will be eligible to participate in the Quarterly Corporate Bonus Plan (the “Bonus Plan”). Your annual incentive bonus target is $500,000. Any bonus is payable at Snowflake’s discretion based upon both Company and individual performance.\n\nYour eligibility and compensation under this Bonus Plan will be governed under the terms of the Bonus Plan and applicable Snowf

## Create a RAG

In [13]:
from snowflake.cortex import complete

class RAG:

    def __init__(self, session):
        self.session = session
        self.retriever = CortexSearchRetriever(snowpark_session=self.session, limit_to_retrieve=4)

    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question in long-form, fully and completely, based on the context. Do not hallucinate.
          If you don´t have the information just say so. If you do have the information you need, just tell me the answer.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        response = ""
        response = complete("mistral-large2", prompt, session = session)
        return response

    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG(session)

In [14]:
response = rag.query("Who is Snowflake's current CRO?")

In [15]:
response

" Michael Gannon is Snowflake's current Chief Revenue Officer (CRO). He was appointed to replace Christopher W. Degnan, who notified the company of his intention to retire in March 2025."